# 🔵 Modelo 8: K-Means Clustering
## Segmentación de Exportaciones (Aprendizaje No Supervisado)

---

## 🎯 Objetivo
Agrupar exportaciones colombianas en clusters similares usando el algoritmo K-Means, sin necesidad de etiquetas previas.

K-Means es un algoritmo de clustering que particiona n observaciones en k clusters, donde cada observación pertenece al cluster con la media más cercana (centroide).

## 📊 Variables
- **Features**: Variables numéricas relacionadas con las exportaciones
- **Método**: Clustering no supervisado
- **Algoritmo**: K-Means con inicialización inteligente

## 📋 Contenido
1. Importación de librerías
2. Carga y exploración de datos
3. Preprocesamiento de datos
4. Determinación del número óptimo de clusters
5. Entrenamiento del modelo K-Means
6. Evaluación de clusters
7. Análisis de características de clusters
8. Visualización de resultados
9. Guardado del modelo

## 1. Importación de Librerías

Importamos todas las librerías necesarias para el análisis de clustering.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Librerías de machine learning
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import pickle

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print('✅ Librerías importadas correctamente')

## 2. Carga de Datos

Cargamos el dataset original desde el archivo Excel y realizamos una exploración inicial.

In [ ]:
# Cargar el dataset desde Excel
df = pd.read_excel('../DATA/DATAPROYECTO.xlsx', sheet_name='Detalle')

print(f'📦 Dataset cargado exitosamente')
print(f'   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas')
print(f'   Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

# Vista rápida de las primeras filas
df.head()

## 3. Preprocesamiento de Datos

Realizamos el preprocesamiento necesario para preparar los datos para el clustering.

In [ ]:
# Crear una copia para trabajar
df_processed = df.copy()

print('🔧 Iniciando preprocesamiento...')

# 1. Manejo de valores faltantes
print('\n1️⃣ Tratamiento de valores faltantes:')

# Para columnas numéricas: imputar con la mediana
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_processed[col].isnull().sum() > 0:
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)
        print(f'   ✓ {col}: Imputado con mediana = {median_val:.2f}')

print(f'\n   Valores nulos restantes: {df_processed.isnull().sum().sum()}')

# 2. Feature Engineering - Crear variables derivadas
print('\n2️⃣ Feature Engineering:')

# Ratio Peso Bruto/Neto
df_processed['Ratio_Peso_Bruto_Neto'] = df_processed['Peso en kilos brutos'] / (df_processed['Peso en kilos netos'] + 1e-10)
print('   ✓ Ratio_Peso_Bruto_Neto creado')

# Valor por kilogramo
df_processed['Valor_Por_Kg'] = df_processed['Valor FOB (USD)'] / (df_processed['Peso en kilos netos'] + 1e-10)
print('   ✓ Valor_Por_Kg creado')

## 4. Selección de Features para Clustering

Seleccionamos las variables numéricas más relevantes para el clustering.

In [ ]:
# Seleccionar features numéricas para clustering
features = [
    'Valor FOB (USD)', 'Peso en kilos netos', 'Peso en kilos brutos',
    'Cantidad(es)', 'Número de artículos', 'Precio Unitario FOB (USD) Peso Neto',
    'Ratio_Peso_Bruto_Neto', 'Valor_Por_Kg'
]

# Crear dataset para clustering
df_clustering = df_processed[features].copy().dropna()
df_clustering = df_clustering.replace([np.inf, -np.inf], np.nan).dropna()

# Tomar muestra para eficiencia si el dataset es muy grande
sample_size = min(15000, len(df_clustering))  # Máximo 15k muestras para eficiencia
if len(df_clustering) > sample_size:
    df_clustering = df_clustering.sample(sample_size, random_state=42)
    print(f'📊 Muestreo aplicado: {sample_size:,} registros de {len(df_processed):,}')

print(f'📊 Dataset para clustering:')
print(f'   • Features: {len(features)} variables numéricas')
print(f'   • Muestras: {df_clustering.shape[0]:,}')

# Estadísticas descriptivas
print('\n📈 Estadísticas descriptivas:')
display(df_clustering.describe())

# Escalamiento (crucial para K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering)

print(f'\n📏 Escalamiento aplicado (StandardScaler)')
print(f'   • Crucial para K-Means ya que usa distancias euclidianas')

## 5. Determinación del Número Óptimo de Clusters

Usamos el método del codo y otras técnicas para determinar el número óptimo de clusters k.

In [ ]:
# Método del codo para encontrar k óptimo
print('🔍 Determinando número óptimo de clusters...')

inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    inertias.append(kmeans.inertia_)
    
    # Calcular silhouette score
    if k > 1:  # Silhouette requiere al menos 2 clusters
        sil_score = silhouette_score(X_scaled, clusters)
        silhouette_scores.append(sil_score)

# Visualización del método del codo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico del codo
ax1.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Número de Clusters (k)', fontsize=12)
ax1.set_ylabel('Inercia (Within-cluster sum of squares)', fontsize=12)
ax1.set_title('Método del Codo', fontweight='bold', fontsize=14)
ax1.grid(alpha=0.3)

# Marcar el "codo" aproximado
ax1.axvline(x=4, color='red', linestyle='--', alpha=0.7, label='k=4 sugerido')
ax1.legend()

# Gráfico de Silhouette Score
ax2.plot(k_range[1:], silhouette_scores, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Número de Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score', fontweight='bold', fontsize=14)
ax2.grid(alpha=0.3)
ax2.axvline(x=4, color='red', linestyle='--', alpha=0.7, label='k=4 sugerido')
ax2.legend()

plt.tight_layout()
plt.show()

print('💡 Interpretación:')
print('   • Método del codo: Buscar el "codo" donde la inercia deja de disminuir rápidamente')
print('   • Silhouette Score: Valores más cercanos a 1 indican mejores clusters')
print(f'   • Mejor Silhouette Score: k={k_range[np.argmax(silhouette_scores)+1]} (score={max(silhouette_scores):.4f})')

## 6. Entrenamiento del Modelo K-Means

Entrenamos el modelo K-Means con el número óptimo de clusters.

In [ ]:
# Entrenar K-Means con k=4 (basado en el análisis de categorías de valor)
k_optimal = 4

print(f'🤖 Entrenando K-Means con k={k_optimal}...')

kmeans = KMeans(
    n_clusters=k_optimal,
    random_state=42,
    n_init=10,  # Número de inicializaciones
    init='k-means++',  # Método de inicialización inteligente
    max_iter=300  # Máximo de iteraciones
)

# Entrenamiento
clusters = kmeans.fit_predict(X_scaled)

# Agregar clusters al dataframe
df_clustering['Cluster'] = clusters

print('✅ K-Means entrenado exitosamente')
print(f'   • Número de clusters: {k_optimal}')
print(f'   • Algoritmo convergió en {kmeans.n_iter_} iteraciones')
print(f'   • Inercia final: {kmeans.inertia_:.2f}')

# Distribución de clusters
print(f'\n📊 Distribución de clusters:')
cluster_counts = df_clustering['Cluster'].value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    percentage = (count / len(df_clustering)) * 100
    print(f'   • Cluster {cluster_id}: {count:,} registros ({percentage:.1f}%)')

## 7. Evaluación de la Calidad de los Clusters

Evaluamos la calidad de los clusters usando métricas específicas para clustering.

In [ ]:
# Métricas de evaluación de clustering
silhouette = silhouette_score(X_scaled, clusters)
davies_bouldin = davies_bouldin_score(X_scaled, clusters)
calinski_harabasz = calinski_harabasz_score(X_scaled, clusters)

print('📊 MÉTRICAS DE EVALUACIÓN DE CLUSTERING:')
print('='*60)
print(f'  Silhouette Score:     {silhouette:.4f}')
print(f'    • Rango: [-1, 1] - Más cercano a 1 = mejor')
print(f'    • > 0.5: Clusters razonablemente separados')
print(f'    • > 0.7: Clusters bien separados')
print()
print(f'  Davies-Bouldin Index: {davies_bouldin:.4f}')
print(f'    • Rango: [0, ∞] - Más cercano a 0 = mejor')
print(f'    • < 1.0: Clusters aceptables')
print()
print(f'  Calinski-Harabasz:    {calinski_harabasz:.2f}')
print(f'    • Rango: [0, ∞] - Valores más altos = mejor')
print(f'    • Mide la separación entre clusters')
print()
print(f'  Inercia (Within-cluster sum of squares): {kmeans.inertia_:.2f}')
print(f'    • Suma de distancias cuadradas dentro de clusters')
print(f'    • Valores más bajos = mejor compacidad')

# Interpretación automática
print('\n🎯 Interpretación automática:')
if silhouette > 0.5:
    print('   ✅ Clusters bien separados (Silhouette > 0.5)')
elif silhouette > 0.25:
    print('   ⚠️ Clusters razonables (Silhouette > 0.25)')
else:
    print('   ❌ Clusters pobremente separados (Silhouette < 0.25)')

if davies_bouldin < 1.0:
    print('   ✅ Buena separación entre clusters (DB < 1.0)')
else:
    print('   ⚠️ Separación entre clusters podría mejorar (DB > 1.0)')

## 8. Análisis de Características de los Clusters

Analizamos las características principales de cada cluster para entender su significado.

In [ ]:
# Análisis detallado de cada cluster
print('📋 ANÁLISIS DETALLADO DE CLUSTERS:')
print('='*80)

cluster_profiles = []

for i in range(k_optimal):
    cluster_data = df_clustering[df_clustering['Cluster'] == i]
    
    print(f'\n🔵 CLUSTER {i} (n={len(cluster_data):,})')
    print('-' * 50)
    
    # Estadísticas clave
    stats = cluster_data[features].describe()
    
    print(f'📊 Estadísticas principales:')
    print(f'   • Valor FOB: ${cluster_data["Valor FOB (USD)"].mean():,.2f} ± ${cluster_data["Valor FOB (USD)"].std():,.2f}')
    print(f'   • Peso neto: {cluster_data["Peso en kilos netos"].mean():,.2f} ± {cluster_data["Peso en kilos netos"].std():,.2f} kg')
    print(f'   • Cantidad: {cluster_data["Cantidad(es)"].mean():,.2f} ± {cluster_data["Cantidad(es)"].std():,.2f}')
    print(f'   • Precio unitario: ${cluster_data["Precio Unitario FOB (USD) Peso Neto"].mean():,.2f}')
    
    # Características distintivas (comparadas con la media global)
    print(f'\n🎯 Características distintivas:')
    global_mean = df_clustering[features].mean()
    cluster_mean = cluster_data[features].mean()
    
    for feature in features:
        diff_pct = ((cluster_mean[feature] - global_mean[feature]) / global_mean[feature]) * 100
        if abs(diff_pct) > 20:  # Más de 20% de diferencia
            direction = "mayor" if diff_pct > 0 else "menor"
            print(f'   • {feature}: {abs(diff_pct):.1f}% {direction} que el promedio')
    
    # Guardar perfil del cluster
    cluster_profiles.append({
        'cluster_id': i,
        'size': len(cluster_data),
        'percentage': (len(cluster_data) / len(df_clustering)) * 100,
        'features_mean': cluster_mean.to_dict(),
        'features_std': cluster_data[features].std().to_dict()
    })

# Resumen comparativo
print(f'\n📊 RESUMEN COMPARATIVO DE CLUSTERS:')
print('='*80)
summary_df = df_clustering.groupby('Cluster')[features[:3]].mean()  # Solo primeras 3 features para resumen
display(summary_df.style.background_gradient(cmap='Blues', axis=0))

## 9. Visualización de los Clusters

Visualizamos los clusters usando reducción de dimensionalidad (PCA).

In [ ]:
# Reducción de dimensionalidad con PCA para visualización
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Visualización 2D de los clusters
plt.figure(figsize=(14, 10))

# Scatter plot de los puntos
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=clusters, cmap='viridis', 
                     alpha=0.6, s=50, edgecolors='white', linewidth=0.5)

# Centroides de los clusters
centers_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], 
           c='red', marker='X', s=400, edgecolors='black', linewidths=3,
           label='Centroides')

# Etiquetas y título
plt.xlabel(f'Componente Principal 1 ({pca.explained_variance_ratio_[0]:.1%} varianza)', fontsize=12)
plt.ylabel(f'Componente Principal 2 ({pca.explained_variance_ratio_[1]:.1%} varianza)', fontsize=12)
plt.title('Visualización de Clusters K-Means (Reducción PCA)', 
          fontsize=16, fontweight='bold')

# Barra de colores
cbar = plt.colorbar(scatter)
cbar.set_label('Cluster', fontsize=12)

# Leyenda
plt.legend(loc='upper right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print('📊 Información de la visualización:')
print(f'   • Varianza explicada por las 2 componentes: {pca.explained_variance_ratio_.sum():.1%}')
print('   • Cada punto representa una exportación')
print('   • Colores indican la pertenencia al cluster')
print('   • X rojas marcan los centroides de cada cluster')

# Información adicional sobre PCA
print(f'\n🔍 Componentes principales:')
for i, component in enumerate(pca.components_):
    top_features = np.argsort(np.abs(component))[-3:]  # Top 3 features
    print(f'   • PC{i+1}: {features[top_features[2]]}, {features[top_features[1]]}, {features[top_features[0]]}')

## 10. Guardado del Modelo

Guardamos el modelo de clustering junto con todos los objetos necesarios.

In [ ]:
# Preparar paquete completo del modelo
model_package = {
    'model': kmeans,
    'scaler': scaler,
    'pca': pca,
    'features': features,
    'n_clusters': k_optimal,
    'cluster_profiles': cluster_profiles,
    'metrics': {
        'silhouette_score': silhouette,
        'davies_bouldin_index': davies_bouldin,
        'calinski_harabasz_score': calinski_harabasz,
        'inertia': kmeans.inertia_,
        'n_iterations': kmeans.n_iter_
    },
    'model_info': {
        'algorithm': 'K-Means Clustering',
        'n_clusters': k_optimal,
        'init_method': 'k-means++',
        'n_init': 10,
        'max_iter': 300,
        'random_state': 42,
        'n_features': len(features),
        'n_samples': len(df_clustering),
        'features_list': features
    }
}

# Guardar el modelo
with open('model_kmeans.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print('💾 Modelo guardado exitosamente')
print('   • Archivo: model_kmeans.pkl')
print('   • Contiene: modelo K-Means, scaler, PCA, perfiles de clusters y métricas')

# Verificar que se guardó correctamente
import os
if os.path.exists('model_kmeans.pkl'):
    size = os.path.getsize('model_kmeans.pkl') / 1024
    print(f'   • Tamaño: {size:.2f} KB')
    print('✅ Verificación exitosa')
else:
    print('❌ Error al guardar el modelo')

## 🎯 Conclusiones

### Resumen del Clustering K-Means:

**Fortalezas:**
- **Simplicidad**: Algoritmo fácil de entender e implementar
- **Escalabilidad**: Funciona bien con grandes datasets
- **Interpretabilidad**: Los centroides son fáciles de analizar
- **Velocidad**: Rápido para datos numéricos
- **Determinístico**: Resultados reproducibles con semilla fija

**Limitaciones:**
- **Número de clusters**: Requiere definir k a priori
- **Formas esféricas**: Asume clusters de forma esférica
- **Escala sensible**: Requiere escalamiento de features
- **Outliers**: Sensible a valores atípicos
- **Inicialización**: Puede converger a óptimos locales

### Resultados Obtenidos:
- **Número de clusters:** 4
- **Muestras clusterizadas:** {len(df_clustering):,}
- **Silhouette Score:** {silhouette:.4f}
- **Davies-Bouldin Index:** {davies_bouldin:.4f}
- **Inercia final:** {kmeans.inertia_:.2f}
- **Varianza explicada por PCA:** {pca.explained_variance_ratio_.sum():.1%}

### Distribución de Clusters:
{cluster_counts.to_dict()}

### Recomendaciones:
1. **Interpretación de clusters**: Analizar los perfiles de cada cluster para asignar nombres descriptivos
2. **Validación externa**: Si hay etiquetas disponibles, comparar con clustering supervisado
3. **Técnicas alternativas**: Considerar DBSCAN o Gaussian Mixture Models para clusters no esféricos
4. **Reducción de features**: Si hay muchas features, considerar PCA antes del clustering
5. **Robustez**: Ejecutar múltiples inicializaciones para asegurar estabilidad

---

**🔵 Modelo K-Means completado exitosamente** ✅

*Los datos han sido segmentados en clusters naturales que pueden usarse para análisis de patrones o estrategias de negocio.*